In [2]:
"""
Script Simple de Web Scraping - Protestas Tec Morelia
Busca información real en la web sobre el tema
"""

import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import time
import json

# ============================================================================
# CONFIGURACIÓN
# ============================================================================

QUERY = "Tecnológico Morelia protestas estudiantes 2025"
OUTPUT_FILE = f"informacion_tec_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"

# ============================================================================
# BUSCAR CON DUCKDUCKGO (No requiere API)
# ============================================================================

def buscar_duckduckgo(query, max_results=20):
    """Buscar en DuckDuckGo y obtener resultados reales"""
    print(f"🔍 Buscando en DuckDuckGo: {query}")
    
    url = "https://html.duckduckgo.com/html/"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    data = {
        'q': query,
        'b': '',
        'kl': 'mx-es'
    }
    
    resultados = []
    
    try:
        response = requests.post(url, headers=headers, data=data, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontrar resultados
        results_divs = soup.find_all('div', class_='result')
        
        for result in results_divs[:max_results]:
            try:
                # Título y link
                title_elem = result.find('a', class_='result__a')
                if title_elem:
                    title = title_elem.get_text().strip()
                    link = title_elem.get('href', '')
                    
                    # Snippet/descripción
                    snippet_elem = result.find('a', class_='result__snippet')
                    snippet = snippet_elem.get_text().strip() if snippet_elem else ""
                    
                    if link and title:
                        resultados.append({
                            'titulo': title,
                            'url': link,
                            'descripcion': snippet,
                            'fuente': 'DuckDuckGo',
                            'fecha': datetime.now().strftime('%Y-%m-%d %H:%M')
                        })
                        print(f"  ✓ {title[:70]}...")
            except Exception as e:
                continue
        
        print(f"  Total encontrados: {len(resultados)}")
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
    
    return resultados

# ============================================================================
# OBTENER CONTENIDO DE PÁGINAS
# ============================================================================

def extraer_contenido_pagina(url):
    """Extraer el contenido principal de una página web"""
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remover scripts y estilos
        for script in soup(['script', 'style', 'nav', 'footer', 'header']):
            script.decompose()
        
        # Intentar encontrar el contenido principal
        main_content = None
        
        # Buscar en diferentes contenedores comunes
        for selector in ['article', 'main', '.content', '.entry-content', '.post-content']:
            main_content = soup.select_one(selector)
            if main_content:
                break
        
        # Si no encuentra contenedor, usar todo el body
        if not main_content:
            main_content = soup.find('body')
        
        if main_content:
            # Extraer texto
            texto = main_content.get_text(separator=' ', strip=True)
            # Limpiar espacios múltiples
            texto = ' '.join(texto.split())
            return texto[:2000]  # Primeros 2000 caracteres
        
    except Exception as e:
        return f"Error al extraer: {str(e)}"
    
    return ""

# ============================================================================
# BUSCAR EN SITIOS ESPECÍFICOS DE NOTICIAS
# ============================================================================

def buscar_sitio_especifico(sitio_url, sitio_nombre, keywords):
    """Buscar en un sitio específico"""
    print(f"\n📰 Buscando en {sitio_nombre}...")
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    resultados = []
    
    try:
        response = requests.get(sitio_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Buscar todos los enlaces
        enlaces = soup.find_all('a', href=True)
        
        for enlace in enlaces:
            texto = enlace.get_text().strip()
            href = enlace.get('href', '')
            
            # Filtrar por palabras clave
            if texto and any(keyword.lower() in texto.lower() for keyword in keywords):
                # Asegurar URL completa
                if href.startswith('/'):
                    href = sitio_url + href
                elif not href.startswith('http'):
                    href = sitio_url + '/' + href
                
                resultados.append({
                    'titulo': texto,
                    'url': href,
                    'descripcion': '',
                    'fuente': sitio_nombre,
                    'fecha': datetime.now().strftime('%Y-%m-%d %H:%M')
                })
                print(f"  ✓ {texto[:70]}...")
        
        print(f"  Total encontrados: {len(resultados)}")
        
    except Exception as e:
        print(f"  ✗ Error en {sitio_nombre}: {e}")
    
    return resultados

# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================

def recopilar_informacion():
    """Recopilar información de múltiples fuentes"""
    
    print("="*80)
    print("  🔍 RECOPILADOR DE INFORMACIÓN WEB - PROTESTAS TEC MORELIA")
    print("="*80)
    
    todos_resultados = []
    
    # 1. Buscar en DuckDuckGo
    print("\n" + "─"*80)
    print("BÚSQUEDA 1: DuckDuckGo")
    print("─"*80)
    
    resultados_ddg = buscar_duckduckgo(QUERY, max_results=15)
    todos_resultados.extend(resultados_ddg)
    time.sleep(2)
    
    # 2. Buscar variaciones del tema
    print("\n" + "─"*80)
    print("BÚSQUEDA 2: Búsquedas relacionadas")
    print("─"*80)
    
    queries_relacionadas = [
        "Tec Morelia Patricia Calderón",
        "Instituto Tecnológico Morelia protesta 2025",
        "Estudiantes ITM Michoacán"
    ]
    
    for query in queries_relacionadas:
        print(f"\n🔍 Buscando: {query}")
        resultados = buscar_duckduckgo(query, max_results=5)
        todos_resultados.extend(resultados)
        time.sleep(2)
    
    # 3. Sitios de noticias locales específicos
    print("\n" + "─"*80)
    print("BÚSQUEDA 3: Medios locales")
    print("─"*80)
    
    sitios = [
        ("https://www.lavozdemichoacan.com.mx", "La Voz de Michoacán"),
        ("https://www.quadratin.com.mx/michoacan", "Quadratín"),
        ("https://www.mimorelia.com", "Mi Morelia"),
    ]
    
    keywords = ["tecnológico", "tec", "morelia", "protesta", "estudiantes", 
                "patricia", "calderón", "ITM"]
    
    for url, nombre in sitios:
        resultados_sitio = buscar_sitio_especifico(url, nombre, keywords)
        todos_resultados.extend(resultados_sitio)
        time.sleep(2)
    
    # 4. Remover duplicados
    print("\n" + "─"*80)
    print("PROCESANDO RESULTADOS")
    print("─"*80)
    
    # Eliminar duplicados por URL
    urls_vistas = set()
    resultados_unicos = []
    
    for item in todos_resultados:
        if item['url'] not in urls_vistas:
            urls_vistas.add(item['url'])
            resultados_unicos.append(item)
    
    print(f"✓ {len(resultados_unicos)} resultados únicos encontrados")
    
    # 5. Intentar extraer contenido de las páginas principales
    print("\n" + "─"*80)
    print("EXTRAYENDO CONTENIDO (esto puede tomar un momento)")
    print("─"*80)
    
    for i, item in enumerate(resultados_unicos[:10], 1):  # Solo primeros 10
        print(f"  [{i}/10] Extrayendo: {item['titulo'][:50]}...")
        contenido = extraer_contenido_pagina(item['url'])
        item['contenido'] = contenido
        time.sleep(1)
    
    # 6. Guardar resultados
    print("\n" + "─"*80)
    print("GUARDANDO RESULTADOS")
    print("─"*80)
    
    if resultados_unicos:
        # Guardar CSV
        with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
            fieldnames = ['titulo', 'url', 'descripcion', 'fuente', 'fecha', 'contenido']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            
            for item in resultados_unicos:
                if 'contenido' not in item:
                    item['contenido'] = ''
                writer.writerow(item)
        
        print(f"\n✅ ¡Listo! {len(resultados_unicos)} resultados guardados en: {OUTPUT_FILE}")
        
        # Guardar también en JSON para mejor legibilidad
        json_file = OUTPUT_FILE.replace('.csv', '.json')
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(resultados_unicos, f, ensure_ascii=False, indent=2)
        
        print(f"✅ También guardado en formato JSON: {json_file}")
        
        # Mostrar resumen
        print("\n📊 RESUMEN POR FUENTE:")
        from collections import Counter
        fuentes = Counter([item['fuente'] for item in resultados_unicos])
        for fuente, count in fuentes.most_common():
            print(f"  • {fuente}: {count} resultados")
        
        # Mostrar primeros 5 resultados
        print("\n📰 PRIMEROS 5 RESULTADOS:")
        for i, item in enumerate(resultados_unicos[:5], 1):
            print(f"\n  {i}. {item['titulo']}")
            print(f"     🔗 {item['url']}")
            if item['descripcion']:
                print(f"     📝 {item['descripcion'][:100]}...")
    
    else:
        print("\n⚠️ No se encontraron resultados")
    
    print("\n" + "="*80)
    print("  ✅ PROCESO COMPLETADO")
    print("="*80)
    print("\n💡 Puedes abrir el archivo CSV con Excel o el JSON con cualquier editor")

# ============================================================================
# EJECUTAR
# ============================================================================

if __name__ == "__main__":
    recopilar_informacion()


"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
INSTRUCCIONES:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Instalar:
   pip install requests beautifulsoup4

2. Ejecutar:
   python scraper_web.py

3. Resultados:
   - informacion_tec_XXXXXX.csv (para Excel)
   - informacion_tec_XXXXXX.json (más legible)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
QUÉ HACE:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✓ Busca en DuckDuckGo (no requiere API)
✓ Busca variaciones del tema
✓ Scrapea medios locales de Michoacán
✓ Extrae contenido real de las páginas
✓ Genera CSV y JSON con toda la información
✓ URLs funcionan correctamente
✓ Sin necesidad de redes sociales

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

  🔍 RECOPILADOR DE INFORMACIÓN WEB - PROTESTAS TEC MORELIA

────────────────────────────────────────────────────────────────────────────────
BÚSQUEDA 1: DuckDuckGo
────────────────────────────────────────────────────────────────────────────────
🔍 Buscando en DuckDuckGo: Tecnológico Morelia protestas estudiantes 2025
  ✓ Por inseguridad y descuido del campus, alumnos del Tec de Morelia ......
  ✓ Estudiantes del Tecnológico de Morelia se manifiestan ante inseguridad...
  ✓ Agencia Mich - ⚠️ Toman estudiantes las instalaciones ... - Facebook...
  ✓ Toman instalaciones del Tec de Morelia; jóvenes piden mejorar ......
  ✓ Choques, detenciones y gas lacrimógeno en segundo día de protestas en ...
  ✓ Asaltos, secuestros y un caso de violación: denuncian estudiantes del ...
  ✓ Escalan protestas contra violencia en la entidad...
  ✓ Escalan protestas en Michoacán; estudiantes se ... - La Jornada...
  ✓ Estudiantes de Morelia y Uruapan marchan para exigir justicia por el ....
  ✓ Estudiantes d

'\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\nINSTRUCCIONES:\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n\n1. Instalar:\n   pip install requests beautifulsoup4\n\n2. Ejecutar:\n   python scraper_web.py\n\n3. Resultados:\n   - informacion_tec_XXXXXX.csv (para Excel)\n   - informacion_tec_XXXXXX.json (más legible)\n\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\nQUÉ HACE:\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n\n✓ Busca en DuckDuckGo (no requiere API)\n✓ Busca variaciones del tema\n✓ Scrapea medios locales de Michoacán\n✓ Extrae contenido real de las páginas\n✓ Genera CSV y JSON con toda la información\n✓ URLs funcionan correctamente\n✓ Sin necesidad de redes sociales\n\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n'